In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Piecewise, Objective, SolverFactory, Param, NonNegativeReals, value, exp
from thermo import Chemical
import numpy as np

def optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names):
    model = ConcreteModel()

    # def initial_concentration_setup(model, compound_names):
    #     total_reactants = sum(comp['stoichiometry'] for comp in compound_names['reactants'].values())
    #     total_products = sum(comp['stoichiometry'] for comp in compound_names['products'].values())
    #     model.C_A = Var(domain=NonNegativeReals, initialize=model.C_A_in * compound_names['reactants']['A']['stoichiometry'] / total_reactants)
    #     model.C_B = Var(domain=NonNegativeReals, initialize=model.C_B_in * compound_names['reactants']['B']['stoichiometry'] / total_reactants)
    #     model.C_C = Var(domain=NonNegativeReals, initialize=0)  # Assuming no product in the feed
    #     model.C_D = Var(domain=NonNegativeReals, initialize=0)  # Assuming no product in the feed
    #     return model.C_A, model.C_B, model.C_C, model.C_D

    def reaction_enthalpy(compound_names):
        delta_H_rxn_react = 0.0
        delta_H_rxn_prod = 0.0
        for compound in compound_names['reactants'].values():
            chemr = Chemical(compound['formula'])
            delta_H_rxn_react += compound['stoichiometry'] * chemr.Hf
        for compound in compound_names['products'].values():
            chemp = Chemical(compound['formula'])
            delta_H_rxn_prod += compound['stoichiometry'] * chemp.Hf
        return (delta_H_rxn_prod - delta_H_rxn_react)

    def specific_heat(compound_formula, model):
        compound = Chemical(compound_formula)
        T = model.T_in.value  # Use the inlet temperature as the reference temperature for simplicity

        # Ensure the compound is at the temperature we're interested in
        compound.calculate(T)

        # Determine the phase at the given temperature
        phase = compound.phase

        # Get specific heat capacity based on the phase
        if phase == 's':  # Solid
            Cp = compound.HeatCapacitySolid(T) if compound.HeatCapacitySolid(T) else 0
        elif phase == 'l':  # Liquid
            Cp = compound.HeatCapacityLiquid(T) if compound.HeatCapacityLiquid(T) else 0
        elif phase == 'g':  # Gas
            Cp = compound.HeatCapacityGas(T) if compound.HeatCapacityGas(T) else 0
        else:
            Cp = 0  # Fallback if phase data is unavailable

        return Cp  # Return the computed specific heat value

    R = 8.314  # Gas constant J/(mol K)
    model.v0 = Var(domain=NonNegativeReals, initialize=initial_conditions.get('v0')) if 'v0' in initial_conditions else Var(domain=NonNegativeReals) # Volumetric flow rate
    model.C_A_in = Param(domain=NonNegativeReals, initialize=initial_conditions.get('C_A_in')) if 'C_A_in' in initial_conditions else Var(domain=NonNegativeReals) #Inlet concentration of A
    model.C_B_in = Param(domain=NonNegativeReals, initialize=initial_conditions.get('C_B_in')) if 'C_B_in' in initial_conditions else Var(domain=NonNegativeReals) #Inlet concentration of B
    model.C_C_in = Param(initialize=0)
    model.C_D_in = Param(initialize=0)
    model.V = Param(initialize=initial_conditions.get('V'), domain=NonNegativeReals) if 'V' in initial_conditions else Var(domain=NonNegativeReals)
    model.T_in = Var(initialize=initial_conditions.get('T_in'), domain=NonNegativeReals) if 'T_in' in initial_conditions else Var(initialize=303, domain=NonNegativeReals)
    model.T = Var(domain=NonNegativeReals, initialize=model.T_in)
    model.T_ref = Param(initialize=303)
    # model.C_A, model.C_B, model.C_C, model.C_D = initial_concentration_setup(model, compound_names)
    model.C_A = Var(domain=NonNegativeReals, initialize=0)
    model.C_B = Var(domain=NonNegativeReals, initialize=0)
    model.C_C = Var(domain=NonNegativeReals, initialize=0)
    model.C_D = Var(domain=NonNegativeReals, initialize=0)
    model.Q_dot = Param(initialize=0)  # Heat input/output
    model.W_dot = Param(initialize=0)  # Work input/output


    # Update constraints to use these new initialized values.


    model.k0 = Param(initialize=rate_constant_303K)  # Reaction rate constant
    model.Ea = Param(initialize=activation_energy)  # Activation energy

    for compound_key, compound_data in {**compound_names['reactants'], **compound_names['products']}.items():
        setattr(model, f'SH_{compound_key}', specific_heat(compound_data['formula'], model))

    model.delta_H_rx = Param(initialize=reaction_enthalpy(compound_names))

    model.k = model.k0 * exp((-model.Ea / (R))*((1/model.T) - (1/model.T_ref))) 

    def reaction_rate_expression(model):
        return model.k0 * exp((-model.Ea / (R))*((1/model.T) - (1/model.T_ref))) * (model.C_A) * (model.C_B)
    
    model.reaction_rate = Var(domain=NonNegativeReals)
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == reaction_rate_expression(model))

    def mole_balance_ruleA(model):
        return ((model.C_A_in*model.v0 - model.C_A*model.v0 + (-1*model.reaction_rate) * model.V)/model.V) == 0
    model.mole_balance_A = Constraint(rule=mole_balance_ruleA)

    def mole_balance_ruleB(model):
        return ((model.C_B_in*model.v0 - model.C_B*model.v0 + (-1*model.reaction_rate) * model.V)/model.V) == 0
    model.mole_balance_B = Constraint(rule=mole_balance_ruleB)

    def mole_balance_ruleC(model):
        return ((model.C_C_in*model.v0 - model.C_C*model.v0 + model.reaction_rate * model.V)/model.V) == 0
    model.mole_balance_C = Constraint(rule=mole_balance_ruleC)

    def mole_balance_ruleD(model):
        return ((model.C_D_in*model.v0 - model.C_D*model.v0 + model.reaction_rate * model.V)/model.V) == 0
    model.mole_balance_D = Constraint(rule=mole_balance_ruleD)

    def energy_balance_rule(model):
        Cp_A = model.SH_A
        Cp_B = model.SH_B
        Cp_C = model.SH_C
        Cp_D = model.SH_D

        # Heat capacity flow term for inlet streams (A and B)
        heat_capacity_flow_inlet = -1*(model.v0*model.C_A_in*Cp_A + model.v0*model.C_B_in*Cp_B + model.v0*model.C_C_in*Cp_C + model.v0*model.C_D_in*Cp_D) * (model.T - model.T_in)

        # Reaction enthalpy term
        reaction_enthalpy_term = -1*model.delta_H_rx * model.reaction_rate * model.V

        # Sum of molar flow rates times heat capacities
        total_molar_flow_heat_capacity = (model.V*model.C_A*Cp_A + model.V*model.C_B*Cp_B + model.V*model.C_C*Cp_C + model.V*model.C_D*Cp_D)

        # Energy balance equation set to zero for steady state
        return ((model.Q_dot - model.W_dot + heat_capacity_flow_inlet*reaction_enthalpy_term) / total_molar_flow_heat_capacity) == 0

    model.energy_balance = Constraint(rule=energy_balance_rule)

    def objective_rule(model):
        return (model.C_A_in - model.C_A) / model.C_A_in
    model.objective = Objective(rule=objective_rule, sense=1)  # sense=1 for maximization
    
    solver = SolverFactory('ipopt')
    results = solver.solve(model)

    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('C_A_in:', value(model.C_A_in))
        print('C_B_in:', value(model.C_B_in))
        print('V:', value(model.V))
        print('v0:', value(model.v0))
        print('T_in:', value(model.T_in))
    else:
        print('No optimal solution found.')

    return model

compound_names = {
    'reactants': {
        'A': {'formula': 'C4H8O2', 'stoichiometry': 1},  # Ethyl Acetate
        'B': {'formula': 'NaOH', 'stoichiometry': 1},     # Sodium Hydroxide
    },
    'products': {
        'C': {'formula': 'C2H5OH', 'stoichiometry': 1},  # Ethanol
        'D': {'formula': 'NaC2H3O2', 'stoichiometry': 1},   # Sodium Acetate
    }
}

# compound_names = {
#     'reactants': {
#         'A': {'formula': 'CH3COOH', 'stoichiometry': 1},  # Acetic Acid
#         'B': {'formula': 'C2H5OH', 'stoichiometry': 1},     # Ethanol
#     },
#     'products': {
#         'C': {'formula': 'CH3COOCH2CH3', 'stoichiometry': 1},  # Ethyl Acetate
#         'D': {'formula': 'H2O', 'stoichiometry': 1},   # Water
#     }
# }

# Example usage of the optimization function
initial_conditions = {
    'C_A_in': 1.0,  # Inlet flow rate of A
    'C_B_in': 1.0,  # Inlet flow rate of B
    #'v0': 0.1,
    'V': 100,        # Reactor volume
    #'T_in': 398,    # Inlet temperature of the reactor
}
rate_constant_303K = 0.15 # Example rate constant at 303K
activation_energy = 48.4*1000   # Example activation energy in kJ/mol

# Call the function
model = optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names)

In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Piecewise, Objective, SolverFactory, Param, NonNegativeReals, value, exp
from thermo import Chemical
import numpy as np

def optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names):
    model = ConcreteModel()

    def reaction_enthalpy(compound_names):
        delta_H_rxn_react = 0.0
        delta_H_rxn_prod = 0.0
        for compound in compound_names['reactants'].values():
            chemr = Chemical(compound['formula'])
            delta_H_rxn_react += compound['stoichiometry'] * chemr.Hf
        for compound in compound_names['products'].values():
            chemp = Chemical(compound['formula'])
            delta_H_rxn_prod += compound['stoichiometry'] * chemp.Hf
        return (delta_H_rxn_prod - delta_H_rxn_react)

    def specific_heat(compound_formula, model):
        compound = Chemical(compound_formula)
        T = model.T_in.value  # Use the inlet temperature as the reference temperature for simplicity

        # Ensure the compound is at the temperature we're interested in
        compound.calculate(T)

        # Determine the phase at the given temperature
        phase = compound.phase

        # Get specific heat capacity based on the phase
        if phase == 's':  # Solid
            Cp = compound.HeatCapacitySolid(T) if compound.HeatCapacitySolid(T) else 0
        elif phase == 'l':  # Liquid
            Cp = compound.HeatCapacityLiquid(T) if compound.HeatCapacityLiquid(T) else 0
        elif phase == 'g':  # Gas
            Cp = compound.HeatCapacityGas(T) if compound.HeatCapacityGas(T) else 0
        else:
            Cp = 0  # Fallback if phase data is unavailable

        return Cp  # Return the computed specific heat value


    R = 8.314  # Gas constant J/(mol K)
    model.v0 = Param(domain=NonNegativeReals, initialize=initial_conditions.get('v0')) if 'v0' in initial_conditions else Var(domain=NonNegativeReals) # Volumetric flow rate
    model.C_A_in = Param(domain=NonNegativeReals, initialize=initial_conditions.get('C_A_in')) if 'C_A_in' in initial_conditions else Var(domain=NonNegativeReals) #Inlet concentration of A
    model.C_B_in = Param(domain=NonNegativeReals, initialize=initial_conditions.get('C_B_in')) if 'C_B_in' in initial_conditions else Var(domain=NonNegativeReals) #Inlet concentration of B
    model.C_C_in = Param(initialize=0)
    model.C_D_in = Param(initialize=0)
    model.V = Param(initialize=initial_conditions.get('V'), domain=NonNegativeReals) if 'V' in initial_conditions else Var(domain=NonNegativeReals)
    model.T_in = Var(initialize=initial_conditions.get('T_in'), domain=NonNegativeReals) if 'T_in' in initial_conditions else Var(initialize=303, domain=NonNegativeReals)
    model.T = Var(domain=NonNegativeReals, initialize=model.T_in)
    model.T_ref = Param(initialize=303)
    model.C_A = Var(domain=NonNegativeReals, initialize=model.C_A_in)
    model.C_B = Var(domain=NonNegativeReals, initialize=model.C_B_in)
    model.C_C = Var(domain=NonNegativeReals, initialize=0)
    model.C_D = Var(domain=NonNegativeReals, initialize=0)
    model.Q_dot = Param(initialize=0)  # Heat input/output
    model.W_dot = Param(initialize=0)  # Work input/output
    model.k0 = Param(initialize=rate_constant_303K)  # Reaction rate constant
    model.Ea = Param(initialize=activation_energy)  # Activation energy

    for compound_key, compound_data in {**compound_names['reactants'], **compound_names['products']}.items():
        compound_formula = compound_data['formula']
        # Get the specific heat value
        Cp_value = specific_heat(compound_formula, model)
        # Create the parameter for the specific heat
        Cp_name = f'Cp_{compound_key}'
        setattr(model, Cp_name, Param(initialize=Cp_value, mutable=False, domain=NonNegativeReals))



    model.delta_H_rx = Param(initialize=reaction_enthalpy(compound_names))

    model.k = model.k0 * exp((-model.Ea / (R))*((1/model.T) - (1/model.T_ref))) 

    def reaction_rate_expression(model):
        return model.k0 * exp((-model.Ea / (R))*((1/model.T) - (1/model.T_ref))) * (model.C_A) * (model.C_B)
    
    model.reaction_rate = Var()
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == reaction_rate_expression(model))

    def mole_balance_ruleA(model):
        return ((model.C_A_in*model.v0 - model.C_A*model.v0 + (-1*model.reaction_rate) * model.V)/model.V) == 0
    model.mole_balance_A = Constraint(rule=mole_balance_ruleA)

    def mole_balance_ruleB(model):
        return ((model.C_B_in*model.v0 - model.C_B*model.v0 + (-1*model.reaction_rate) * model.V)/model.V) == 0
    model.mole_balance_B = Constraint(rule=mole_balance_ruleB)

    def mole_balance_ruleC(model):
        return ((model.C_C_in*model.v0 - model.C_C*model.v0 + model.reaction_rate * model.V)/model.V) == 0
    model.mole_balance_C = Constraint(rule=mole_balance_ruleC)

    def mole_balance_ruleD(model):
        return ((model.C_D_in*model.v0 - model.C_D*model.v0 + model.reaction_rate * model.V)/model.V) == 0
    model.mole_balance_D = Constraint(rule=mole_balance_ruleD)

    def energy_balance_rule(model):
        # Example of using specific heat capacities in energy balance
        heat_capacity_flow_inlet = sum(
            model.v0 * getattr(model, f'Cp_{key}').value * model.C_A_in * comp['stoichiometry']
            for key, comp in compound_names['reactants'].items()
        )
        heat_capacity_flow_products = sum(
            model.v0 * getattr(model, f'Cp_{key}').value * model.C_A_in * comp['stoichiometry']
            for key, comp in compound_names['products'].items()
        )

        # Adjust the following to your actual model details
        return model.Q_dot + heat_capacity_flow_inlet - heat_capacity_flow_products == 0

    model.energy_balance = Constraint(rule=energy_balance_rule)



    def objective_rule(model):
        return (model.C_A_in - model.C_A) / model.C_A_in
    model.objective = Objective(rule=objective_rule, sense=1)  # sense=1 for maximization
    
    solver = SolverFactory('ipopt')
    results = solver.solve(model)

    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('C_A_in:', value(model.C_A_in))
        print('C_B_in:', value(model.C_B_in))
        print('V:', value(model.V))
        print('v0:', value(model.v0))
        print('T_in:', value(model.T_in))
    else:
        print('No optimal solution found.')

    return model

compound_names = {
    'reactants': {
        'A': {'formula': 'C4H8O2', 'stoichiometry': 1},  # Ethyl Acetate
        'B': {'formula': 'NaOH', 'stoichiometry': 1},     # Sodium Hydroxide
    },
    'products': {
        'C': {'formula': 'C2H5OH', 'stoichiometry': 1},  # Ethanol
        'D': {'formula': 'NaC2H3O2', 'stoichiometry': 1},   # Sodium Acetate
    }
}

# compound_names = {
#     'reactants': {
#         'A': {'formula': 'CH3COOH', 'stoichiometry': 1},  # Acetic Acid
#         'B': {'formula': 'C2H5OH', 'stoichiometry': 1},     # Ethanol
#     },
#     'products': {
#         'C': {'formula': 'CH3COOCH2CH3', 'stoichiometry': 1},  # Ethyl Acetate
#         'D': {'formula': 'H2O', 'stoichiometry': 1},   # Water
#     }
# }

# Example usage of the optimization function
initial_conditions = {
    'C_A_in': 1.0,  # Inlet flow rate of A
    'C_B_in': 1.0,  # Inlet flow rate of B
    #'v0': 0.1,
    'V': 100,        # Reactor volume
    #'T_in': 398,    # Inlet temperature of the reactor
}
rate_constant_303K = 0.15 # Example rate constant at 303K
activation_energy = 48.4  # Example activation energy in kJ/mol

# Call the function
model = optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names)


In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Piecewise, Objective, SolverFactory, Param, NonNegativeReals, value, exp
from thermo import Chemical
import numpy as np

def optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names):
    model = ConcreteModel()

    def reaction_enthalpy():
        delta_H_rxn_react = 0.0
        delta_H_rxn_prod = 0.0
        for compound in compound_names['reactants'].values():
            chemr = Chemical(compound['formula'])
            delta_H_rxn_react += compound['stoichiometry'] * chemr.Hf
        for compound in compound_names['products'].values():
            chemp = Chemical(compound['formula'])
            delta_H_rxn_prod += compound['stoichiometry'] * chemp.Hf
        return (delta_H_rxn_prod - delta_H_rxn_react)

    def specific_heat(compound_formula):
        compound = Chemical(compound_formula)
        compound.calculate(model.T_in())
        Cp = compound.Cp
        return Cp


    R = 8.314  # Gas constant J/(mol K)
    model.v0 = Param(within=NonNegativeReals, initialize=initial_conditions.get('v0')) if 'v0' in initial_conditions else Var(within=NonNegativeReals) # Volumetric flow rate
    # Declare C_A_in and C_B_in as Var
    model.C_A_in = Var(within=NonNegativeReals, initialize=initial_conditions.get('C_A_in'))
    model.C_B_in = Var(within=NonNegativeReals, initialize=initial_conditions.get('C_B_in'))

    # Initialize C_A and C_B with C_A_in and C_B_in
    model.C_A = Var(within=NonNegativeReals, initialize=model.C_A_in)
    model.C_B = Var(within=NonNegativeReals, initialize=model.C_B_in)
    model.C_C_in = Param(initialize=0)
    model.C_D_in = Param(initialize=0)
    model.V = Param(initialize=initial_conditions.get('V'), within=NonNegativeReals) if 'V' in initial_conditions else Var(within=NonNegativeReals)
    model.T_in = Var(initialize=initial_conditions.get('T_in'), within=NonNegativeReals) if 'T_in' in initial_conditions else Var(initialize=303, within=NonNegativeReals)
    model.T = Var(within=NonNegativeReals, initialize=model.T_in)
    model.T_ref = Param(initialize=303)
    # model.C_A = Var(within=NonNegativeReals, initialize=model.C_A_in)
    # model.C_B = Var(within=NonNegativeReals, initialize=model.C_B_in)
    model.C_C = Var(within=NonNegativeReals, initialize=0)
    model.C_D = Var(within=NonNegativeReals, initialize=0)
    model.Q_dot = Param(initialize=0)  # Heat input/output
    model.W_dot = Param(initialize=0)  # Work input/output
    model.k0 = Param(initialize=rate_constant_303K)  # Reaction rate constant
    model.Ea = Param(initialize=activation_energy)  # Activation energy

    model.k = model.k0 * exp((-model.Ea / R) * ((1 / model.T) - (1 / model.T_in)))

    model.reaction_rate = Var()
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == model.k * model.C_A * model.C_B)

    model.energy_balance = Constraint(expr=sum(
        model.v0 * specific_heat(comp['formula']) * getattr(model, f'C_{key}')
        for key, comp in {**compound_names['reactants'], **compound_names['products']}.items()
    ) == 0)

    def mole_balance_ruleA(model):
        return ((model.C_A_in*model.v0 - model.C_A*model.v0 + (-1*model.reaction_rate) * model.V)/model.V) == 0
    model.mole_balance_A = Constraint(rule=mole_balance_ruleA)

    def mole_balance_ruleB(model):
        return ((model.C_B_in*model.v0 - model.C_B*model.v0 + (-1*model.reaction_rate) * model.V)/model.V) == 0
    model.mole_balance_B = Constraint(rule=mole_balance_ruleB)

    def mole_balance_ruleC(model):
        return ((model.C_C_in*model.v0 - model.C_C*model.v0 + model.reaction_rate * model.V)/model.V) == 0
    model.mole_balance_C = Constraint(rule=mole_balance_ruleC)

    def mole_balance_ruleD(model):
        return ((model.C_D_in*model.v0 - model.C_D*model.v0 + model.reaction_rate * model.V)/model.V) == 0
    model.mole_balance_D = Constraint(rule=mole_balance_ruleD)



    def objective_rule(model):
        return (model.C_A_in - model.C_A) / model.C_A_in
    model.objective = Objective(rule=objective_rule, sense=1)  # sense=1 for maximization
    
    solver = SolverFactory('ipopt')
    results = solver.solve(model)

    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('C_A_in:', value(model.C_A_in))
        print('C_B_in:', value(model.C_B_in))
        print('V:', value(model.V))
        print('v0:', value(model.v0))
        print('T_in:', value(model.T_in))
    else:
        print('No optimal solution found.')

    return model

compound_names = {
    'reactants': {
        'A': {'formula': 'C4H8O2', 'stoichiometry': 1},  # Ethyl Acetate
        'B': {'formula': 'NaOH', 'stoichiometry': 1},     # Sodium Hydroxide
    },
    'products': {
        'C': {'formula': 'C2H5OH', 'stoichiometry': 1},  # Ethanol
        'D': {'formula': 'NaC2H3O2', 'stoichiometry': 1},   # Sodium Acetate
    }
}

# compound_names = {
#     'reactants': {
#         'A': {'formula': 'CH3COOH', 'stoichiometry': 1},  # Acetic Acid
#         'B': {'formula': 'C2H5OH', 'stoichiometry': 1},     # Ethanol
#     },
#     'products': {
#         'C': {'formula': 'CH3COOCH2CH3', 'stoichiometry': 1},  # Ethyl Acetate
#         'D': {'formula': 'H2O', 'stoichiometry': 1},   # Water
#     }
# }

# Example usage of the optimization function
initial_conditions = {
    'C_A_in': 1.0,  # Inlet flow rate of A
    'C_B_in': 1.0,  # Inlet flow rate of B
    #'v0': 0.1,
    'V': 105,        # Reactor volume
    #'T_in': 398,    # Inlet temperature of the reactor
}
rate_constant_303K = 0.15 # Example rate constant at 303K
activation_energy = 48.4*1000  # Example activation energy in kJ/mol

# Call the function
model = optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names)


In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Objective, SolverFactory, Param, NonNegativeReals, exp
from pyomo.environ import value as pyomo_value
from thermo import Chemical

def optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names):
    model = ConcreteModel()

    # Parameters or variables depending on if they are provided in initial_conditions
    model.v0 = Var(within=NonNegativeReals, initialize=initial_conditions.get('v0')) if 'v0' in initial_conditions else Var(within=NonNegativeReals)
    model.C_A_in = Var(within=NonNegativeReals, initialize=initial_conditions.get('C_A_in')) if 'C_A_in' in initial_conditions else Var(within=NonNegativeReals)
    model.C_B_in = Var(within=NonNegativeReals, initialize=initial_conditions.get('C_B_in')) if 'C_B_in' in initial_conditions else Var(within=NonNegativeReals)
    model.C_C_in = Param(initialize=0)
    model.C_D_in = Param(initialize=0)
    model.V = Param(initialize=initial_conditions.get('V'), within=NonNegativeReals) if 'V' in initial_conditions else Var(domain=NonNegativeReals)
    model.T_in = Var(initialize=initial_conditions.get('T_in'), within=NonNegativeReals) if 'T_in' in initial_conditions else Var(initialize=303, domain=NonNegativeReals)
    model.T = Var(within=NonNegativeReals, initialize=model.T_in)
    model.T_ref = Param(initialize=303)
    model.C_A = Var(within=NonNegativeReals, initialize=model.C_A_in)
    model.C_B = Var(within=NonNegativeReals, initialize=model.C_B_in)
    model.C_C = Var(within=NonNegativeReals, initialize=0)
    model.C_D = Var(within=NonNegativeReals, initialize=0)
    model.Q_dot = Param(initialize=0)
    model.W_dot = Param(initialize=0)
    model.k0 = Param(initialize=rate_constant_303K)
    model.Ea = Param(initialize=activation_energy)

    # Reaction enthalpy calculation
    def reaction_enthalpy():
        delta_H_rxn_react = 0.0
        delta_H_rxn_prod = 0.0
        for compound in compound_names['reactants'].values():
            chemr = Chemical(compound['formula'])
            delta_H_rxn_react += compound['stoichiometry'] * chemr.Hf
        for compound in compound_names['products'].values():
            chemp = Chemical(compound['formula'])
            delta_H_rxn_prod += compound['stoichiometry'] * chemp.Hf
        return delta_H_rxn_prod - delta_H_rxn_react

    model.delta_H_rx = Param(initialize=reaction_enthalpy())

    # Reaction rate expression
    model.k = model.k0 * exp((-model.Ea / (8.314)) * ((1 / model.T) - (1 / model.T_ref)))
    model.reaction_rate = Var()
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == model.k * model.C_A * model.C_B)

    # Constraints for mole balances
    def mole_balance_ruleA(model):
        return (model.C_A_in * model.v0 - model.C_A * model.v0 - model.reaction_rate * model.V) / model.V == 0
    model.mole_balance_A = Constraint(rule=mole_balance_ruleA)

    def mole_balance_ruleB(model):
        return (model.C_B_in * model.v0 - model.C_B * model.v0 - model.reaction_rate * model.V) / model.V == 0
    model.mole_balance_B = Constraint(rule=mole_balance_ruleB)

    # Objective function to maximize the conversion of A
    model.objective = Objective(expr=(model.C_A_in - model.C_A) / model.C_A_in, sense=1)

    # Solve the model using IPOPT solver
    solver = SolverFactory('ipopt')
    results = solver.solve(model)

    # Output the results
    if results.solver.status == 'ok' and results.solver.termination_condition == 'optimal':
        print('Found optimal solution.')
        print('C_A_in:', pyomo_value(model.C_A_in))
        print('C_B_in:', pyomo_value(model.C_B_in))
        print('T:', pyomo_value(model.T))
    else:
        print('No optimal solution found.')

    return model

# Example input data
initial_conditions = {
    'T_in': 350,  # Example temperature in K
    #'C_A_in': 1.2,  # Example concentration of A in mol/L
    #'C_B_in': 1.5  # Example concentration of B in mol/L
}
rate_constant_303K = 0.15  # Example rate constant at 303 K
activation_energy = 48000  # Example activation energy in J/mol

compound_names = {
    'reactants': {
        'A': {'formula': 'C4H8O2', 'stoichiometry': 1},
        'B': {'formula': 'NaOH', 'stoichiometry': 1},
    },
    'products': {
        'C': {'formula': 'C2H5OH', 'stoichiometry': 1},
        'D': {'formula': 'NaC2H3O2', 'stoichiometry': 1},
    }
}

# Run the optimization
model = optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names)


In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Objective, SolverFactory, Param, NonNegativeReals, exp
from pyomo.environ import value as pyomo_value
from thermo import Chemical

def optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names):
    model = ConcreteModel()

    def specific_heat(compound_formula, model):
        compound = Chemical(compound_formula)
        compound.calculate(model.T_in.value)  # Use the inlet temperature for calculations

        # Determine the phase and use appropriate specific heat capacity
        phase = compound.phase
        if phase == 's':
            Cp = compound.HeatCapacitySolid if compound.HeatCapacitySolid else 0
        elif phase == 'l':
            Cp = compound.HeatCapacityLiquid if compound.HeatCapacityLiquid else 0
        elif phase == 'g':
            Cp = compound.HeatCapacityGas if compound.HeatCapacityGas else 0
        else:
            Cp = 0  # Fallback if phase data is unavailable

        return Cp

    # Constants
    R = 8.314  # Gas constant J/(mol K)

    # Parameters from initial conditions
    model.C_A_in = Param(initialize=initial_conditions['C_A_in'], within=NonNegativeReals)
    model.C_B_in = Param(initialize=initial_conditions['C_B_in'], within=NonNegativeReals)
    model.V = Param(initialize=initial_conditions['V'], within=NonNegativeReals)

    # Variable for optimization
    model.T_in = Var(within=NonNegativeReals, initialize=initial_conditions.get('T_in', 303), bounds=(273, 500))

    # Reaction rate dependent on temperature
    model.k0 = Param(initialize=rate_constant_303K)
    model.Ea = Param(initialize=activation_energy)
    model.T_ref = Param(initialize=303)  # Reference temperature
    model.k = model.k0 * exp((-model.Ea / R) * ((1 / model.T_in) - (1 / model.T_ref)))

    model.reaction_rate = Var(domain=NonNegativeReals)
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == model.k * model.C_A_in * model.C_B_in)

    # Specific heats are dependent on temperature
    for compound_key, compound_data in {**compound_names['reactants'], **compound_names['products']}.items():
        setattr(model, f'Cp_{compound_key}', specific_heat(compound_data['formula'], model))

    # Objective function: For example, maximize or minimize temperature
    model.objective = Objective(expr=model.T_in, sense=-1)  # Sense -1 for maximization, could be 1 for minimization

    # Solver configuration and solve
    solver = SolverFactory('ipopt')
    results = solver.solve(model)

    # Check and print results
    if results.solver.status == 'ok' and results.solver.termination_condition == 'optimal':
        print('Found optimal solution.')
        print('Optimal Inlet Temperature (T_in):', pyomo_value(model.T_in))
        print('Optimal Volumetric Flow Rate (v0):', pyomo_value(model.v0))
    else:
        print('No optimal solution found.')

    return model

compound_names = {
    'reactants': {
        'A': {'formula': 'C4H8O2', 'stoichiometry': 1},  # Ethyl Acetate
        'B': {'formula': 'NaOH', 'stoichiometry': 1},     # Sodium Hydroxide
    },
    'products': {
        'C': {'formula': 'C2H5OH', 'stoichiometry': 1},  # Ethanol
        'D': {'formula': 'NaC2H3O2', 'stoichiometry': 1},   # Sodium Acetate
    }
}

# compound_names = {
#     'reactants': {
#         'A': {'formula': 'CH3COOH', 'stoichiometry': 1},  # Acetic Acid
#         'B': {'formula': 'C2H5OH', 'stoichiometry': 1},     # Ethanol
#     },
#     'products': {
#         'C': {'formula': 'CH3COOCH2CH3', 'stoichiometry': 1},  # Ethyl Acetate
#         'D': {'formula': 'H2O', 'stoichiometry': 1},   # Water
#     }
# }

# Example usage of the optimization function
initial_conditions = {
    'C_A_in': 1.0,  # Inlet flow rate of A
    'C_B_in': 1.0,  # Inlet flow rate of B
    #'v0': 0.1,
    'V': 100,        # Reactor volume
    'T_in': 398,    # Inlet temperature of the reactor
}
rate_constant_303K = 0.15 # Example rate constant at 303K
activation_energy = 48.4*1000  # Example activation energy in kJ/mol


# Call the function
model = optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names)
